In [ ]:
import pandas as pd
import numpy as np
import requests
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df = pd.read_csv('aviation_csv.csv', delimiter=',', encoding='cp1252', low_memory=False)
df = df[(df.event_year >= 1982) & (df.event_year <= 2007)]

In [ ]:
df.Latitude = pd.to_numeric(df.Latitude, errors = 'coerce')
df.Longitude = pd.to_numeric(df.Longitude, errors = 'coerce')

In [ ]:
df.columns = df.columns.str.replace('.','_')
df.columns = df.columns.str.lower()
df.rename(columns={'total_uninjured': 'uninjured',
                   'total_minor_injuries': 'minor_inj',
                   'total_serious_injuries': 'serious_inj',
                   'total_fatal_injuries': 'fatal_inj',
                   'broad_phase_of_flight' : 'flight_phase',
                   'phase_of_flight' : 'flight_phase',
                   'purpose_of_flight' : 'flight_purpose',
                   'amateur_built' : 'is_amateur_built'}, inplace = True)

df.drop(columns={'injury_severity',
                 'publication_date',
                 'airport_code',
                 'airport_name',
                 'air_carrier',
                 'report_status'}, inplace = True)


In [ ]:
df['is_fatal'] = df['fatal_inj'].fillna(0).apply(lambda record: record >= 1)
df[['fatal_inj', 'serious_inj', 'minor_inj', 'uninjured']].fillna(0)
def convert_amateur_built(record):
    if record == 'Y' or record == 'Yes':
        return True
    else:
        return False

df['is_amateur_built'] = df['is_amateur_built'].apply(convert_amateur_built)

In [ ]:
df['flight_phase'].fillna('Unknown')
df['location'] = df['location'].fillna('NO DATA')
df['location'] = df['location'].str.upper()

In [ ]:
# Fixing latidude and longitude
new_lat_list = []
new_lng_list = []
geometry_cache = {}
api_counter = 0
already_entered = 0
error_counter = 0
max = 0 # Change value

API_KEY = os.environ.get('GOOGLE_API_KEY')

# Iterate over latitude, longitude, and location in parallel
for original_lat, original_lng, location in zip(df.latitude, df.longitude, df.location):

    while api_counter < max:  # Don't go broke

        if location not in geometry_cache:  # Check if location is not in the cache

            if pd.isnull(original_lat):  # Check if original latitude is missing

                params = {
                    'key': API_KEY,
                    'address': location
                }
                base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'

                # Send geocoding request and get response
                response = requests.get(base_url, params=params).json()

                if response['status'] == 'OK':  # If geocoding request is successful

                    geometry = response['results'][0]['geometry']
                    new_lat = geometry['location']['lat']
                    new_lng = geometry['location']['lng']
                    api_counter += 1  # Increment API call counter

                else:  # If geocoding request encounters an error
                    new_lat, new_lng = 'Error', 'Error'
                    error_counter += 1  # Increment error counter

            else:  # If original latitude is present
                new_lat = original_lat
                new_lng = original_lng
                already_entered += 1  # Increment already entered counter

            geometry_cache[location] = [new_lat, new_lng]  # Cache the new latitude and longitude

        elif location in geometry_cache:  # If location is already in the cache
            new_lat = geometry_cache[location][0]  # Retrieve latitude from the cache
            new_lng = geometry_cache[location][1]  # Retrieve longitude from the cache

        new_lat_list.append(new_lat)  # Append new latitude to the list
        new_lng_list.append(new_lng)  # Append new longitude to the list

print(f'{api_counter} API calls')  # Print the number of API calls made
print(f'{already_entered} already in the database')  # Print the number of locations already present
print(f'{error_counter} errors')  # Print the number of errors encountered

In [ ]:
df.insert(6,'lat', new_lat_list)
df.insert(7,'lng', new_lng_list)

In [ ]:
df_export = df.copy()
df_export.drop(columns={'latitude', 'longitude'}, inplace=True)
df_export.to_csv('cleaned_data.csv', index=False)